In [20]:
#in my Windows the optimizers are located in
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/tensorflow/python/keras
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\tensorflow\python\keras
#/c/Users/zb0857/AppData/Local/Continuum/anaconda2/envs/Tensorflow/lib/site-packages/sklearn/neural_network
#C:\Users\zb0857\AppData\Local\Continuum\anaconda3\envs\ML\Lib\site-packages\sklearn\neural_network
#rememmber to start the notebook using the environment tensorflow


from numpy.random import seed
import numpy as np
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

import tensorflow as tf
# from tensorflow.python.keras.optimizers import Fire
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import optimizers

from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import log_loss

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torchsummary import summary


import matplotlib.pyplot as plt
%matplotlib inline


#it seems that cateorical cross-entropy is not available in pytorch. We have to make or own
#    implementation of the loss function
#https://datascience.stackexchange.com/questions/55962/pytorch-doing-a-cross-entropy-loss-when-the-predictions-already-have-probabiliti
def categorical_cross_entropy(y_pred, y_true):
    y_pred = torch.clamp(y_pred, 1e-9, 1 - 1e-9)
    return -(y_true * torch.log(y_pred)).sum(dim=1).mean()


### CrossEntropyLoss
$\text{loss}(x, class) = -\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}\right)$
                       $= -x[class] + \log\left(\sum_j \exp(x[j])\right)$
                       
                       
        nll_loss(log_softmax(input, 1),


### Softmax
$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$

## Preparing the input data

In [21]:
#Preparing the input data

#loading the MNIST dataset
mnist = tf.keras.datasets.mnist

#Separating into train and test (60000 train, 10000 test)
(x_train, y_train0),(x_test, y_test0) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#flattening the images (from 28x28 to 784)
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)


#converting the otputs (labels) into one hot vectors
lb = preprocessing.LabelBinarizer()
lb.fit(np.array([0,1,2,3,4,5,6,7,8,9]))
y_train = lb.transform(y_train0)
y_test = lb.transform(y_test0)


## Defining the input parameters


In [22]:
#Defining the input parameters
BatchSize         = 5001
NeuronsLayer1     = 100
NeuronsLayer2     = 100
Epochs            = 100

## Torch by me
https://stackoverflow.com/questions/51041128/pytorch-predict-single-example

In [23]:
# Create Tensors to hold inputs and outputs

torch_X_train = torch.from_numpy(x_train)
torch_y_train = torch.from_numpy(y_train)

#the trainloader helps us deal with the batches
train = torch.utils.data.TensorDataset(torch_X_train.float(),torch_y_train.float())
train_loader = torch.utils.data.DataLoader(train, batch_size = BatchSize, shuffle = False)


In [24]:
# Defining the net

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(784,NeuronsLayer1)
        self.linear2 = nn.Linear(NeuronsLayer1,NeuronsLayer2)
        self.linear3 = nn.Linear(NeuronsLayer2,10)
    
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = F.softmax(self.linear3(X), dim=1)
        return X



In [25]:
# Construct our model by instantiating the class defined above.
model = MLP()


# Construct our loss function and an Optimizer. 

# loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = torch.nn.NLLLoss()
# loss_fn = categorical_cross_entropy()




optimizer = torch.optim.Adam(model.parameters())


In [26]:
#Show aprameters of the network
model.parameters

<bound method Module.parameters of MLP(
  (linear1): Linear(in_features=784, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=100, bias=True)
  (linear3): Linear(in_features=100, out_features=10, bias=True)
)>

In [27]:
#train the network

for epoch in range(Epochs):
    loss_avg = 0.0
    N_elements = 0
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(X_batch)

        # Compute loss
        loss = categorical_cross_entropy(y_pred, y_batch)

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_avg += loss.item()*X_batch.size()[0]
        N_elements += X_batch.size()[0]

    # print loss after each epoch    
    print(epoch+1, loss.item(), loss_avg/N_elements)


1 1.850341796875 2.121517493289709
2 1.0361303091049194 1.4550372563242913
3 0.5268484950065613 0.7938365449965
4 0.356965571641922 0.5048875432923436
5 0.29270148277282715 0.4006705708324909
6 0.25832346081733704 0.3512985180899501
7 0.2352951169013977 0.3205249663233757
8 0.2181140035390854 0.29769819027110933
9 0.2041122019290924 0.2789956733778119
10 0.19231396913528442 0.26285469793751837
11 0.18144388496875763 0.24817762532830237
12 0.17134718596935272 0.2345243138000369
13 0.1618925780057907 0.22164313690960408
14 0.1532825231552124 0.2097093479819596
15 0.14553293585777283 0.19860787144228817
16 0.13840030133724213 0.18846724583879113
17 0.13191820681095123 0.17926902678906917
18 0.12607821822166443 0.1708042189307511
19 0.12064962089061737 0.16299815270379187
20 0.11586832255125046 0.15587103875987232
21 0.11133965104818344 0.14930231818668543
22 0.10712460428476334 0.14317228548787533
23 0.10316768288612366 0.13745868683904408
24 0.09963034093379974 0.1321375793594867
25 0.09

In [28]:
#Predicting
y_train_hat_Pytorch = model(torch_X_train.float()).detach().numpy()

print(log_loss(y_train, y_train_hat_Pytorch))
# print(categorical_cross_entropy(torch.from_numpy(y_train).float(), torch.from_numpy(y_train_hat_Pytorch)))

0.01680797794812698


In [10]:
print(y_train[0])
print(y_train[1])

[0 0 0 0 0 1 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0]


In [11]:
print(y_train_hat_Pytorch[0])
print(y_train_hat_Pytorch[1])

[1.6251083e-16 1.0013642e-09 2.1500260e-12 2.6722231e-03 1.0064816e-22
 9.9732780e-01 4.1204474e-20 6.0437831e-12 3.5702879e-15 2.8144921e-13]
[9.9999523e-01 1.5031657e-14 4.7886738e-06 1.8797798e-10 2.6785288e-13
 8.6480465e-13 5.7663790e-10 3.5290895e-09 2.2213843e-12 2.8939884e-09]


In [12]:
#Show aprameters of the network

summary(model, (1, 784))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 100]          78,500
            Linear-2               [-1, 1, 100]          10,100
            Linear-3                [-1, 1, 10]           1,010
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.34
Estimated Total Size (MB): 0.35
----------------------------------------------------------------
